In [1]:
import pandas as pd
from collections import defaultdict
import sys

In [2]:
finalfile="/Users/irffanalahi/Research/Research_code/gitignorefolder/MHB_per_read/moretest/readbaseddecon3/data/RealRun/collectmasterdf/tryingtotroubleshoot/comparecpgcoverage/sm100/mincpg3/nocov/BULK-PBMC-1389.bg_howsm_single_mode_pp_masterdf.txt_rej-99999.0_mincpg3_rejmode_nov_final.txt" #"toydata/BULK-PBMC-1389.bg_howsm_single_mode_pp_masterdf.txt_rej-99999.0_mincpg1_rejmode_nov_final.txt"#"/Users/irffanalahi/Research/Research_code/gitignorefolder/MHB_per_read/moretest/readbaseddecon3/data/RealRun/collectmasterdf/sm_globaloutsubset/blwithneu_hypo_stat.txt_0.01_2_g100.txtWithSurrounding.txt_1cpgnc.txt_result/blwithneu_hypo_stat.txt_0.01_2_g100.txtWithSurrounding.txt_1cpgnc.txt_result_masterdf.txttemp_finalized/nocov_mincpg1/BULK-PBMC-1389.bg_howsm_single_mode_pp_masterdf.txt_rej-99999.0_mincpg1_rejmode_nov_final.txt"
sm="/Users/irffanalahi/Research/Research_code/gitignorefolder/MHB_per_read/moretest/readbaseddecon3/data/RealRun/sm/blwithneu_hypo_stat.txt_0.01_2_g100.txtWithSurrounding.txt_1cpgnc.txt"
#celllist=["'CD14'","'CD19'","'CD4'","'CD8'","'CD56'","'Neu'","'EPCAM'"] 

finaldf=pd.read_csv(finalfile,sep="\t")
finaldf['acceptedCpG'] = finaldf.acceptedCpG.apply(lambda x: x[1:-1].split(','))
finaldf['notacceptedCpG'] = finaldf.notacceptedCpG.apply(lambda x: x[1:-1].split(','))
finaldf['CheckedCpG'] = finaldf.CheckedCpG.apply(lambda x: x[1:-1].split(','))
finaldf['finalrejectedfor']=finaldf.finalrejectedfor.apply(lambda x: x[1:-1].split(','))

smdf=pd.read_csv(sm,sep="\t")
smdf['celltype']=smdf.celltype.apply(lambda x: x[1:-1].split(','))
finaldf.head()

,ReadName,allcelltype,ucelltype,assignedcelltype,CheckedCpG,mismatchCpG,notacceptedCpG,acceptedCpG,finalacceptedfor,finalrejectedfor,MateCIGARok
0,A00580:76:HFLG2DSXX:2:2252:8015:27680,[],[''],lowmapq,[],[''],[''],[''],notdetermined,[otdetermine],Y
1,A00118:116:HJ7VFDSXX:3:1502:16947:25895,[],[''],lowmapq,[],[''],[''],[''],notdetermined,[otdetermine],Y
2,A00580:76:HFLG2DSXX:4:1137:7853:36104,[],[''],lowmapq,[],[''],[''],[''],notdetermined,[otdetermine],Y
3,A00580:76:HFLG2DSXX:2:1623:3920:33458,[],[''],NotAssigned,[],[''],[''],[''],notdetermined,[otdetermine],Y
4,A00118:116:HJ7VFDSXX:3:2539:32226:16063,[],[''],lowmapq,[],[''],[''],[''],notdetermined,[otdetermine],Y


In [3]:
smdf.head()

,chrom,start,end,celltype,blockchr,blockstart,blockend
0,chr15,52052102,52052104,['CD14'],chr15,52051953,52052253
1,chr3,122060546,122060548,['CD14'],chr3,122060397,122060721
2,chr16,79097508,79097510,['CD14'],chr16,79097359,79097659
3,chr1,159431113,159431115,['CD14'],chr1,159430964,159431264
4,chr12,48336372,48336374,['CD14'],chr12,48336223,48336523


In [4]:
celllist=(smdf['celltype']).tolist()
celllist=[x for sublist in celllist for x in sublist]
celllist=list(set(celllist))
print(celllist)

["'CD14'", "'Neu'", "'CD19'", "'CD4'", "'CD56'", "'CD8'"]


In [5]:
############# needs checking############


def accfeatures(fdf,cells):
    outdict={}
    for cell in cells:
        
        celldf=fdf[fdf['finalacceptedfor']==cell]
        correscpg=(celldf['acceptedCpG']).tolist()
        correscpg=list(map(frozenset,correscpg))
        
        correscpg=list(set(correscpg))
        outdict[cell]=correscpg
    return outdict

    
def rejfeatures(fdf,sdf,cells):
    outdict={}
    
    
    for cell in cells:   
        #print(" \""+cell+"\"")
        negmask=fdf['finalrejectedfor'].apply(lambda x: ("\""+cell+"\"" in x) or (" \""+cell+"\"" in x)) ###check
        
        neg=fdf[negmask]
        
        
        correscpg=(neg['notacceptedCpG']).tolist()
        
        correscpg=realCorresCpg(correscpg,sdf,cell)
        
        correscpg=list(map(frozenset,correscpg))
        
        correscpg=list(set(correscpg))
        outdict[cell]=correscpg
 
    return outdict

        
def realCorresCpg(correscpg,sdf,cell):
    realcorres=[]
    for cpglist in correscpg:
        temp=[]
        for ccpg in cpglist:
            
           
                
            eccpg=eval(ccpg)
            
            if not eccpg:  #####################################check thoroughly
                continue
          
            
            chrom,start=eccpg.split(":")
          
            
            fromsm=sdf[(sdf['chrom']==chrom) & (sdf['start']==int(start))]
            if fromsm.shape[0]!=1:
                print("error. Exiting")
                sys.exit(1)
            
            celltype=fromsm['celltype']
            celltype=[x for sublist in celltype for x in sublist]
            if len(celltype)!=1:
                print("wrong.Exiting")
                sys.exit(1)
            
            if cell==celltype[0]:
                temp.append(ccpg)
           
        if len(temp)!=0:
            realcorres.append(temp)
    
    return realcorres
            
def corealgo(fdf,sdf,cells):
    accfeaturedict=accfeatures(fdf,cells)
    rejfeaturedict=rejfeatures(fdf,sdf,cells)
    
    allfrag={}  ### feature
    allfraglen={}  ###feature len
    accfraglen={} ###feature len
    rejfraglen={} ####feature len
    
    for cell in accfeaturedict:
        
        accfraglen[cell]=len(accfeaturedict[cell])
        
        rejfraglen[cell]=len(rejfeaturedict[cell])
        
        totalfeature=accfeaturedict[cell]+rejfeaturedict[cell]
        
        
        
        totaluniquefeature=list(set(totalfeature))
        #print(len(totaluniquefeature))  ##### some exclusive positive feature?
        
        allfrag[cell]=totaluniquefeature
        allfraglen[cell]=len(totaluniquefeature)
        
    
    
    
    with open(finalfile+"_posfeature.txt", 'w') as f:
        print(accfeaturedict, file=f)
    
    with open(finalfile+"_posfeaturelen.txt", 'w') as f:
        print(accfraglen, file=f)
        
    with open(finalfile+"_negfeature.txt", 'w') as f:
        print(rejfeaturedict, file=f)
        
    
    with open(finalfile+"_negfeaturelen.txt", 'w') as f:
        print(rejfraglen, file=f)
        
    
    with open(finalfile+"_allfeature.txt", 'w') as f:
        print(allfrag, file=f)
        
    with open(finalfile+"_allfeaturelen.txt", 'w') as f:
        print(allfraglen, file=f)
    
        
    return allfrag,allfraglen,accfraglen,rejfraglen
    
       
            
    
    
    
    
    
    
    
    

In [6]:
allfrag,allfraglen,accfraglen,rejfraglen=corealgo(finaldf,smdf,celllist)

print(allfraglen,accfraglen,rejfraglen)

{"'CD14'": 173, "'Neu'": 210, "'CD19'": 304, "'CD4'": 223, "'CD56'": 274, "'CD8'": 275} {"'CD14'": 59, "'Neu'": 10, "'CD19'": 28, "'CD4'": 30, "'CD56'": 57, "'CD8'": 19} {"'CD14'": 140, "'Neu'": 204, "'CD19'": 289, "'CD4'": 206, "'CD56'": 237, "'CD8'": 263}
